<a href="https://colab.research.google.com/github/DurgaBhavana/5731Submissions/blob/master/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [107]:
abstract_text = []
count = 0
for i in range(10):
  link = 'https://citeseerx.ist.psu.edu/search;jsessionid=87FF6C66EA09F22314C131669600CF98?q=natural+language+processing&t=doc&sort=rlv&start='
  page = requests.get(link + str (count))
  abstract = (BeautifulSoup(page.text, 'html.parser')).find_all(class_='pubabstract')
  count = count + 10
  for i in abstract:
    abstract_text.append(i.text.replace('\n', '').strip())

In [108]:
df = pd.DataFrame((abstract_text), columns =['Abstract'])
df

,Abstract
0,Abstract not found
1,describe a method for statistical modeling bas...
2,Scaling conditional random fields for natural ...
3,The paper addresses the issue of cooperation b...
4,In most natural language processing applicatio...
...,...
95,This paper presents a workbench built by Pribe...
96,Abstract—Natural Language Processing (NLP) is ...
97,"ABSTRACT: After twenty years of disfavor, a te..."
98,Text statistics are frequently used in stylome...


#Text Prepocessing

In [109]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from nltk.stem import PorterStemmer
from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [123]:
# Special characters removal
df['After noise removal'] = df['Abstract'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
# Punctuation removal
df['Punctuation removal'] = df['After noise removal'].str.replace('[^\w\s]','')
# Remove numbers
df['Remove numbers'] = df['Punctuation removal'].str.replace('\d+', '')
# Stopwords removal
stop_word = stopwords.words('english')
df['Stopwords removal'] = df['Remove numbers'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_word))
# Lower Casing
df['Lower casing'] = df['Stopwords removal'].apply(lambda x: " ".join(x.lower() for x in x.split()))
# Stemming
st = PorterStemmer()
df['Stemming'] = df['Lower casing'].apply(lambda x: " ".join([st.stem(word) for word in x]))
# Lemmatization
df['Lemmatization'] = df['Stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [125]:
final_df = df


# **INFO5731 Assignment Three**


In this assignment, you are required to conduct information extraction, semantic analysis based on the dataset you collected from assignment two. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**


(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the noun phrases and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets).

In [111]:
# 1.1
nltk.download('punkt')
import itertools
list_of_words = []
for i in df['Lower casing']:
  list_of_words.append(nltk.tokenize.word_tokenize(i))
words = [i for i in list_of_words if i != []]
looping = list(itertools.chain.from_iterable(words))
#N-grams (N=3 -> Trigrams)
fd = nltk.FreqDist(nltk.trigrams(looping))
fd.most_common()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[(('natural', 'language', 'processing'), 76),
 (('language', 'processing', 'nlp'), 21),
 (('language', 'processing', 'systems'), 5),
 (('abstract', 'natural', 'language'), 4),
 (('sense', 'id', 'sense'), 4),
 (('language', 'processing', 'the'), 3),
 (('language', 'processing', 'we'), 3),
 (('language', 'processing', 'techniques'), 3),
 (('processing', 'nlp', 'tasks'), 3),
 (('lex', 'sign', 'sense'), 3),
 (('sign', 'sense', 'id'), 3),
 (('id', 'sense', 'id'), 3),
 (('in', 'natural', 'language'), 2),
 (('language', 'processing', 'applications'), 2),
 (('neural', 'network', 'architecture'), 2),
 (('language', 'processing', 'tasks'), 2),
 (('part', 'speech', 'tagging'), 2),
 (('using', 'natural', 'language'), 2),
 (('natural', 'languages', 'languages'), 2),
 (('languages', 'languages', 'spoken'), 2),
 (('statistical', 'natural', 'language'), 2),
 (('information', 'retrieval', 'system'), 2),
 (('in', 'paper', 'describe'), 2),
 (('processing', 'nlp', 'techniques'), 2),
 (('this', 'paper', 'r

In [112]:
# 1.2
# Bigrams
fd = nltk.FreqDist(nltk.bigrams(looping))
fd_dict = dict(fd)
for bi_gram in fd:
  print(str(bi_gram) + '-' + str(fd_dict[bi_gram] / looping.count(bi_gram[1])))

('abstract', 'found')-1.0
('found', 'describe')-0.1111111111111111
('describe', 'method')-0.2
('method', 'statistical')-0.125
('statistical', 'modeling')-0.3333333333333333
('modeling', 'based')-0.07692307692307693
('based', 'maximum')-0.3333333333333333
('maximum', 'entropy')-1.0
('entropy', 'we')-0.041666666666666664
('we', 'present')-0.5
('present', 'maximum')-0.3333333333333333
('maximum', 'likelihood')-0.5
('likelihood', 'approach')-0.09090909090909091
('approach', 'automatically')-0.3333333333333333
('automatically', 'constructing')-1.0
('constructing', 'maximum')-0.3333333333333333
('entropy', 'models')-0.25
('models', 'describe')-0.1111111111111111
('describe', 'implement')-1.0
('implement', 'approach')-0.09090909090909091
('approach', 'efficiently')-1.0
('efficiently', 'using')-0.09090909090909091
('using', 'examples')-1.0
('examples', 'several')-0.2
('several', 'problems')-0.3333333333333333
('problems', 'natural')-0.010416666666666666
('natural', 'language')-0.81651376146788

In [113]:
from textblob import TextBlob

In [114]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [115]:
def get_sentence():
  s = ''
  for  i in df['Lower casing']:
    s = s + i
  return s

In [116]:
# 1.3
# Extracting nouns
freq = []
phrases = []
s = get_sentence()
for i in df['Lower casing']:
  for values in TextBlob(i).noun_phrases:
    phrases.append(values)
for phrase in phrases:
  f = []
  for j in df['Lower casing']:
    percentage = j.count(phrase)/s.count(phrase)
    f.append(percentage)
  freq.append(f)
np_df = pd.DataFrame(freq).T
np_df.columns = phrases
np_df

,describe method,maximum entropy,present maximum likelihood approach,maximum entropy models describe,natural language processing,conditional random fields,natural language processing terms conditions terms conditions copyright,minerva access,paper addresses issue cooperation linguistics,natural language processing nlp general linguistics machine translation mt,direction cooperation,applications linguistics nlp,natural language processing applications description logics,encode knowledge base syntactic semantic pragmatic elements,drive semantic interpretation,natural language generation processes,description logics,neural network architecture learning algorithm,various natural language processing tasks,part speech,entity recognition semantic role,natural language processing,natural language processing,broad narrow senses,broad sense,processing issues levels,natural language understanding,speech recognition syntactic semantic analysis sentences,robots interact humans,natural language need responsive way humans,language situations,natural language processing system robots performs incremental semantic interpretation,natural languages languages,point languages,natural language processing collection techniques,abstract ambiguity,natural languages,ambiguous computers,language way people,natural language processing nlp,...,interface domain,specific knowledge general linguis tic resources,results experiences,upper model variety applications past years,map nodes nodes,word categories,priori information classes,process model word classes,central topic thesis use som,natural language processing,tica development company,natural language processing technology,linguistic resources software tools,considerable number,practical purposes,natural language processing nlp,effective approach,educational setting,nlp involves,process learning,natural acquisition,educational systems,effective approaches,years disfavor technology,imitates processes brain,natural language experiments sejnowski rosenberg,neural network,language observe rules pronunciation,text statistics,stylometry cryptography studies,paper text statistics tools,iso prolog,natural language processing details,usage user,callable predicates logic limitations,summarize experience,different projects,natural language processing nlp,conclude nlp benefit framenet,different ways sketch problems need
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,1.0,1.0,1.0,1.0,0.013158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013158,0.013158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013158,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.013158,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013158,0.013158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013158,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.013158,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013158,0.013158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013158,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0
4,0.0,0.0,0.0,0.0,0.013158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.5,0.0,0.0,0.0,0.0,0.013158,0.013158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.


# **Question 2: Undersand TF-IDF and Document representation**


(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program:

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [137]:
import math
sentences = final_df["Lower casing"].values.tolist()
words = set([j for i in sentences for j in i.split(" ")])
tf_idf_df = pd.DataFrame(words, columns=["words"])
count=1

def tf_idf(x,sentence):
  value = sentence.count(x)
  size = len(sentence.split(" "))
  if(value != 0):
    return (value/size)*(math.log(size/value, 10))
  else:
    return 0
    
for sentence in sentences:
  tf_idf_df["Review"+str(count)] = tf_idf_df["words"].apply(lambda x: tf_idf(x,sentence))
  count=count+1

In [136]:
tf_idf_df

,words,Review1,Review2,Review3,Review4,Review5,Review6,Review7,Review8,Review9,Review10,Review11,Review12,Review13,Review14,Review15,Review16,Review17,Review18,Review19,Review20,Review21,Review22,Review23,Review24,Review25,Review26,Review27,Review28,Review29,Review30,Review31,Review32,Review33,Review34,Review35,Review36,Review37,Review38,Review39,...,Review61,Review62,Review63,Review64,Review65,Review66,Review67,Review68,Review69,Review70,Review71,Review72,Review73,Review74,Review75,Review76,Review77,Review78,Review79,Review80,Review81,Review82,Review83,Review84,Review85,Review86,Review87,Review88,Review89,Review90,Review91,Review92,Review93,Review94,Review95,Review96,Review97,Review98,Review99,Review100
0,introduced,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.035577,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062963,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1,compound,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.050428,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2,unification,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
3,educational,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.083728,0.0,0.0,0.0
4,central,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.047036,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,research,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.103933,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.051684,0.051684,0.0,0.050428,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.053013,0.0,0.0,0.0,0.081866,0.0,0.0,0.043231,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1114,agents,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1115,start,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000

In [117]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
X_Value = "text directly (rather than e.g. titles and abstracts), and suggests appropriate approaches to doing this, with a focus on the role of natural language processing. The paper also comments on possible connections with data and knowledge retrieval, and concludes by emphasizing the importance of rigorous"
X_after_tokens = word_tokenize(X_Value)
stop = stopwords.words('english') 
X_after_stop_words = {word for word in X_after_tokens if not word in stop}

In [118]:
def cosine_value(X_after_stop_words, Y_after_stop_words):
  cosine = 0
  list1 = []
  list2 = []
  rvector = X_after_stop_words.union(Y_after_stop_words)  
  for w in rvector: 
    if w in X_after_stop_words: list1.append(1)
    else: list1.append(0) 
    if w in X_after_stop_words: list2.append(1) 
    else: list2.append(0) 
  for i in range(len(rvector)): 
    cosine += list1[i]*list2[i] 
  cv = cosine / float((sum(list1)*sum(list2))**0.5)
  return cv

In [119]:
cosine_similarity = []
for i in df['Lower casing']:
  Y_after_tokens = word_tokenize(i)
  Y_after_stop_words = {word for word in Y_after_tokens if not word in stop}
  cosine_similarity.append(cosine_value(X_after_stop_words, Y_after_stop_words))

In [120]:
cs_df = pd.DataFrame(list(zip(df['Lower casing'], cosine_similarity)), columns = ['Abstracts', 'CS'])
cs_df

,Abstracts,CS
0,abstract found,1.0
1,describe method statistical modeling based max...,1.0
2,scaling conditional random fields natural lang...,1.0
3,the paper addresses issue cooperation linguist...,1.0
4,in natural language processing applications de...,1.0
...,...,...
95,this paper presents workbench built priberam i...,1.0
96,abstract natural language processing nlp effec...,1.0
97,abstract after twenty years disfavor technolog...,1.0
98,text statistics frequently used stylometry cry...,1.0



# **Question 3: Create your own training and evaluation data for sentiment analysis**


(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification.

In [121]:
df = pd.DataFrame((abstract_text), columns =['Abstract'])
df.to_csv('Abstracts for Sentiment Analysis.csv')
df

,Abstract
0,Abstract not found
1,describe a method for statistical modeling bas...
2,Scaling conditional random fields for natural ...
3,The paper addresses the issue of cooperation b...
4,In most natural language processing applicatio...
...,...
95,This paper presents a workbench built by Pribe...
96,Abstract—Natural Language Processing (NLP) is ...
97,"ABSTRACT: After twenty years of disfavor, a te..."
98,Text statistics are frequently used in stylome...


File link - 

https://github.com/DurgaBhavana/5731Submissions/blob/master/Abstracts%20for%20Sentiment%20Analysis.csv